In [2]:
!pip install uncertainties

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from uncertainties import ufloat
from uncertainties.umath import sqrt

In [12]:
def remove_outliers(df, cols):
    print(df)
    if len(df[1]) == 1:
        return df

    filtered_df = df.copy()
    for col in cols:
        mu = filtered_df[col].mean()
        sigma = filtered_df[col].std()
        filtered_df = filtered_df[np.abs(filtered_df[col] - mu) <= 2 * sigma]

    return filtered_df
os.makedirs("raw_data", exist_ok=True)

files=os.listdir(os.path.join("raw_data/"))

# [pd.read_csv(files) for file in os.path.join( "C:","616","DataCalculations","DataCalculations","raw_data", file)]
# Load data from CSV files into a list of DataFrames
data = []
for file in files:
    data.append(pd.read_csv(os.path.join("raw_data", file)))

data = [remove_outliers(df, ["neutral_fall_seconds", "charged_rise_seconds"]) for df in data]


   trial  voltage_V  neutral_fall_seconds  charged_rise_seconds  \
0     16      512.3                  9.53                  1.36   

   ionization_seconds       d  rho    g       eta       b         p  room_temp  
0                 0.0  0.0076  886  9.8  0.000018  0.0082  101520.0         21  


KeyError: 1

In [11]:
# Compute charges with uncertainty
charges = []
# Check if 'data' is empty and handle accordingly
if not data:  # or if len(data) == 0
    print("Warning: 'data' is empty. No charges calculated.")
else:
    for df in data:
        d_lines = ufloat(0.5e-3, 0.1e-4)
        vf = d_lines / df["neutral_fall_seconds"].mean()
        vr = d_lines / df["charged_rise_seconds"].mean()

        voltage_error = round(df["voltage_V"].mean() * 0.0009 + 0.2, 1)
        V = ufloat(df["voltage_V"].mean(), voltage_error)
        d = df["d"].mean()
        E = V / d

        eta = df["eta"].mean()
        rho = df["rho"].mean()
        b = df["b"].mean()
        p = ufloat(df["p"].mean(), 1000)

        a = sqrt((b / (2*p))**2 + (9 * eta * vf) / (2 * df["g"].mean() * rho)) - (b / (2*p))
        m = (4/3) * np.pi * a.nominal_value**3 * rho
        q = m * df["g"].mean() * (vf + vr) / (E * vf)

        charges.append(q)

real_charge = 1.602e-19
num_measurements = [len(df["neutral_fall_seconds"]) for df in data]

# Check if charges is empty before proceeding
if charges:
    charge_values = np.array([q.nominal_value for q in charges])
    # Handle potential empty charge_values after filtering:
    if len(charge_values) > 0:
        normalized_charge = charges / min(charge_values)
    else:
        print("Warning: 'charge_values' is empty after filtering. Normalization skipped.")
        normalized_charge = np.nan  # or any other appropriate value

    systematic_uncertainties = np.array([q.std_dev for q in charges])
    statistical_uncertainties = 1e-19 / np.sqrt(num_measurements)
    z_scores = (charge_values / normalized_charge - real_charge) / np.sqrt(statistical_uncertainties**2 + systematic_uncertainties**2)
    chi_squared = np.sum(z_scores ** 2)

    droplet_labels = [chr(65 + (i % 26)) + (chr(65 + (i // 26 - 1)) if i >= 26 else "") for i in range(len(charges))]

    results_df = pd.DataFrame({
        "Droplet": droplet_labels,
        "Charge (C)": np.round(charge_values, 4),
        "# Charges": np.round(normalized_charge, 4),
        "Statistical Uncertainty": np.round(statistical_uncertainties, 4),
        "Systematic Uncertainty": np.round(systematic_uncertainties, 4),
        "Z-Score": np.round(z_scores, 4)
    })

    print(results_df)
    print(f"\nΧ² = {chi_squared}")
else:
    print("Warning: 'charges' is empty. No calculations performed.")

In [ ]:
# Plotting
plt.figure(figsize=(8, 6))
e = min(charge_values)
scale = e * 1e19
y_max_scaled = 1.629e-18 * 1e19 / scale

plt.errorbar(normalized_charge[:6], charge_values[:6] / e, yerr=systematic_uncertainties[:6] / e,
             fmt='o', label="Measured Charges, High Quality Data")
plt.errorbar(normalized_charge[6:], charge_values[6:] / e, yerr=systematic_uncertainties[6:] / e,
             fmt='o', label="Measured Charges, Low Quality Data", alpha=0.8)

plt.scatter(range(1, 8), range(1, 8), label="Millikan's Data", alpha=0.8)
plt.axhline(y=1, color='gray', linestyle='--', alpha=0.3)
plt.xlabel("Number of Charges")
plt.ylabel("Charge (elementary charge)")
plt.legend()
plt.title("Number of Charges vs Charge")
plt.savefig("output_plot.png")
plt.show()